In [1]:
import os
import torch
import torchaudio
from datasets import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
!pip install evaluate
!pip install jiwer
import evaluate
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 80.4 MB/s eta 0:00:00:00:01


2024-08-01 15:37:19.092293: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 15:37:19.092391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 15:37:19.189770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:

# 1. 加载数据
def load_audio_files(audio_dir):
    audio_files = []
    for filename in os.listdir(audio_dir):
        if filename.endswith(".wav"):
            audio_path = os.path.join(audio_dir, filename)
            audio_files.append(audio_path)
    audio_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
    return audio_files

def load_transcripts(transcript_dir):
    transcripts = []
    txt_files = [f for f in os.listdir(transcript_dir) if f.endswith('.txt')]
    txt_files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    for file in txt_files:
        with open(os.path.join(transcript_dir, file), 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read().strip()
            transcripts.append(text)
    return transcripts

def resample_audio(waveform, original_rate, target_rate):
    resampler = torchaudio.transforms.Resample(orig_freq=original_rate, new_freq=target_rate)
    return resampler(waveform)

def create_dataset(audio_files, transcripts):
    data = {"audio": [], "sentence": []}
    for audio_file, transcript in zip(audio_files, transcripts):
        waveform, sample_rate = torchaudio.load(audio_file)
        if sample_rate != 16000:
            waveform = resample_audio(waveform, sample_rate, 16000)
        audio = {"array": waveform.squeeze().numpy(), "sampling_rate": 16000}
        data["audio"].append(audio)
        data["sentence"].append(transcript)
    return Dataset.from_dict(data)

audio_dir = "/kaggle/input/6002project/training_data"
transcript_dir = "/kaggle/input/6002project/reference"

audio_files = load_audio_files(audio_dir)
transcripts = load_transcripts(transcript_dir)
dataset = create_dataset(audio_files, transcripts)

# 2. 划分数据集
total_size = len(dataset)
eval_size = int(0.1 * total_size)

eval_dataset = dataset.select(range(eval_size))

# 3. 加载预训练模型和处理器
processor = WhisperProcessor.from_pretrained("alvanlii/whisper-small-cantonese",language="yue",task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("alvanlii/whisper-small-cantonese")
model.eval()

# 4. 对评估数据集进行转录
def transcribe_audio(batch):
    input_features = processor.feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"], return_tensors="pt").input_features
    input_features = input_features.to(model.device)
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription

eval_dataset = eval_dataset.map(lambda batch: {"prediction": transcribe_audio(batch)}, remove_columns=["audio"])

# 去掉标点符号和空格
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 去掉标点符号
    text = re.sub(r'\s+', '', text)  # 去掉多余的空格
    return text

def split_chars(text):
    return " ".join(list(text))

def tokenize_text(text):
    cleaned_text = clean_text(text)
    tokenized_text = split_chars(cleaned_text)
    return tokenized_text

# 5. 计算WER
predictions = eval_dataset["prediction"]
references = eval_dataset["sentence"]

# 对预测和参考句子进行分词和清理
predictions_tokenized = [tokenize_text(pred) for pred in predictions]
references_tokenized = [tokenize_text(ref) for ref in references]

# 打印分词结果以供检查
for pred_tok, ref_tok in zip(predictions_tokenized, references_tokenized):
    print(f"Tokenized Prediction: {pred_tok}")
    print(f"Tokenized Reference: {ref_tok}")
    print("------")

# 使用 evaluate 计算并输出WER
metric = evaluate.load("wer")
wer_result = metric.compute(predictions=predictions_tokenized, references=references_tokenized)
print(f"WER: {wer_result * 100:.2f}%")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Tokenized Prediction: 倒 消 又 重 補 都 好 再 重 好 多 補
Tokenized Reference: 讀 書 要 從 薄 到 厚 再 從 厚 到 薄
------
Tokenized Prediction: 所 謂 會 讀 書 就 係 本 住 誠 意 虧 度 有 加 之 個 數
Tokenized Reference: 所 謂 會 讀 書 就 係 本 住 誠 意 去 讀 有 價 值 嘅 書
------
Tokenized Prediction: 好 消 死 人 更 動 得 幸 數 人 生
Tokenized Reference: 好 書 使 人 更 懂 得 享 受 人 生
------
Tokenized Prediction: 打 開 你 哋 有 聲 好 笑 我 哋 就 可 以 用 耳 朵 怨 冷 群 笑 聆 聽 中 外 不 同 駐 坐
Tokenized Reference: 打 開 呢 本 有 聲 好 書 我 哋 就 可 以 用 耳 朵 閱 覽 群 書 聆 聽 中 外 不 同 著 作
------
Tokenized Prediction: 香 港 電 台 J Z W O
Tokenized Reference: 香 港 電 臺 製 作
------
Tokenized Prediction: 有 聲 好 笑
Tokenized Reference: 有 聲 好 書
------
Tokenized Prediction: 需 置 痛 癮 啲 嘈 請 老 好 故 事
Tokenized Reference: 最 折 騰 人 的 籌 錢 留 學 故 事
------
Tokenized Prediction: 為 咗 掃 請 六 號 以 利 卡 加 盞 其 中 最 令 人 登 湖 詐 詩 嘅 女 子 係 呢 時 好 家 李 東 風
Tokenized Reference: 為 咗 索 錢 留 學 而 累 及 家 人 其 中 最 令 人 燈 目 咋 舌 嘅 例 子 係 歷 史 學 家 黎 東 方
------
Tokenized Prediction: 佢 令 一 家 人 連 膨 脂 親 友 都 兩 作 一 團
Tokenized Reference: 佢 令 一 家 人 連 旁 支 親 友 都 亂 作 一 團
------
To

In [5]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
# 1. 加载数据
def load_audio_files(audio_dir):
    audio_files = []
    for filename in os.listdir(audio_dir):
        if filename.endswith(".wav"):
            audio_path = os.path.join(audio_dir, filename)
            audio_files.append(audio_path)
    audio_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
    return audio_files

def load_transcripts(transcript_dir):
    transcripts = []
    txt_files = [f for f in os.listdir(transcript_dir) if f.endswith('.txt')]
    txt_files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    for file in txt_files:
        with open(os.path.join(transcript_dir, file), 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read().strip()
            transcripts.append(text)
    return transcripts

def resample_audio(waveform, original_rate, target_rate):
    resampler = torchaudio.transforms.Resample(orig_freq=original_rate, new_freq=target_rate)
    return resampler(waveform)

def create_dataset(audio_files, transcripts):
    data = {"audio": [], "sentence": []}
    for audio_file, transcript in zip(audio_files, transcripts):
        waveform, sample_rate = torchaudio.load(audio_file)
        if sample_rate != 16000:
            waveform = resample_audio(waveform, sample_rate, 16000)
        audio = {"array": waveform.squeeze().numpy(), "sampling_rate": 16000}
        data["audio"].append(audio)
        data["sentence"].append(transcript)
    return Dataset.from_dict(data)

audio_dir = "/kaggle/input/6002project/training_data"
transcript_dir = "/kaggle/input/6002project/reference"

audio_files = load_audio_files(audio_dir)
transcripts = load_transcripts(transcript_dir)
dataset = create_dataset(audio_files, transcripts)

# 2. 划分数据集
total_size = len(dataset)
eval_size = int(0.1 * total_size)

eval_dataset = dataset.select(range(eval_size))

# 3. 加载预训练模型和处理器
#processor = WhisperProcessor.from_pretrained("Scrya/whisper-large-v2-cantonese",language="yue",task="transcribe")
#model = WhisperForConditionalGeneration.from_pretrained("Scrya/whisper-large-v2-cantonese")
processor = AutoProcessor.from_pretrained("Scrya/whisper-large-v2-cantonese")
model = AutoModelForSpeechSeq2Seq.from_pretrained("Scrya/whisper-large-v2-cantonese")
model.eval()

# 4. 对评估数据集进行转录
def transcribe_audio(batch):
    input_features = processor.feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"], return_tensors="pt").input_features
    input_features = input_features.to(model.device)
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription

eval_dataset = eval_dataset.map(lambda batch: {"prediction": transcribe_audio(batch)}, remove_columns=["audio"])

# 去掉标点符号和空格
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 去掉标点符号
    text = re.sub(r'\s+', '', text)  # 去掉多余的空格
    return text

def split_chars(text):
    return " ".join(list(text))

def tokenize_text(text):
    cleaned_text = clean_text(text)
    tokenized_text = split_chars(cleaned_text)
    return tokenized_text

# 5. 计算WER
predictions = eval_dataset["prediction"]
references = eval_dataset["sentence"]

# 对预测和参考句子进行分词和清理
predictions_tokenized = [tokenize_text(pred) for pred in predictions]
references_tokenized = [tokenize_text(ref) for ref in references]

# 打印分词结果以供检查
for pred_tok, ref_tok in zip(predictions_tokenized, references_tokenized):
    print(f"Tokenized Prediction: {pred_tok}")
    print(f"Tokenized Reference: {ref_tok}")
    print("------")

# 使用 evaluate 计算并输出WER
metric = evaluate.load("wer")
wer_result = metric.compute(predictions=predictions_tokenized, references=references_tokenized)
print(f"WER: {wer_result * 100:.2f}%")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Tokenized Prediction: 屌 銷 要 重 寶 都 好 再 重 好 多 寶
Tokenized Reference: 讀 書 要 從 薄 到 厚 再 從 厚 到 薄
------
Tokenized Prediction: 所 謂 未 讀 書 就 係 本 著 心 意 會 讀 有 加 字 嘅 書
Tokenized Reference: 所 謂 會 讀 書 就 係 本 住 誠 意 去 讀 有 價 值 嘅 書
------
Tokenized Prediction: 好 少 死 人 更 動 得 享 受 人 生
Tokenized Reference: 好 書 使 人 更 懂 得 享 受 人 生
------
Tokenized Prediction: 打 開 你 本 有 聲 好 笑 我 哋 就 可 以 用 已 多 元 朗 群 語 聆 聽 中 外 不 同 住 住
Tokenized Reference: 打 開 呢 本 有 聲 好 書 我 哋 就 可 以 用 耳 朵 閱 覽 群 書 聆 聽 中 外 不 同 著 作
------
Tokenized Prediction: 香 港 電 台 在 做
Tokenized Reference: 香 港 電 臺 製 作
------
Tokenized Prediction: 有 聲 好 笑
Tokenized Reference: 有 聲 好 書
------
Tokenized Prediction: 最 悸 痛 人 嘅 惆 情 留 後 故 事
Tokenized Reference: 最 折 騰 人 的 籌 錢 留 學 故 事
------
Tokenized Prediction: 為 咗 熟 悉 老 號 以 利 卡 加 人 其 中 最 令 人 登 模 渣 事 嘅 例 子 係 呢 時 好 家 李 東 峰
Tokenized Reference: 為 咗 索 錢 留 學 而 累 及 家 人 其 中 最 令 人 燈 目 咋 舌 嘅 例 子 係 歷 史 學 家 黎 東 方
------
Tokenized Prediction: 可 憐 一 家 人 連 胖 子 親 友 都 亂 咗 一 頹
Tokenized Reference: 佢 令 一 家 人 連 旁 支 親 友 都 亂 作 一 團
------
Tokeni